In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from glob import glob
from collections import defaultdict
from PIL import Image
import numpy as np
import cv2
from functions import *

In [2]:
import os
os.chdir('C:/Users/zin/CV')

# 학습 데이터의 경로와 정보를 가진 파일의 경로를 설정
traindata_dir = "./data/train"
traindata_info_file = "./data/train.csv"

# 테스트 데이터의 경로와 정보를 가진 파일의 경로를 설정
testdata_dir = "./data/test"
testdata_info_file = "./data/test.csv"

# 학습 데이터의 class, image path, target에 대한 정보가 들어있는 csv파일을 읽기
train_data = pd.read_csv(traindata_info_file)

# 테스트 데이터
test_data = pd.read_csv(testdata_info_file)

# 학습 데이터의 정보를 출력
train_info = train_data.info()
train_head = train_data.head()

train_info, train_head

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15021 entries, 0 to 15020
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   class_name  15021 non-null  object
 1   image_path  15021 non-null  object
 2   target      15021 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 352.2+ KB


(None,
   class_name                image_path  target
 0  n01872401  n01872401/sketch_50.JPEG      59
 1  n02417914  n02417914/sketch_11.JPEG     202
 2  n02106166   n02106166/sketch_3.JPEG     138
 3  n04235860   n04235860/sketch_2.JPEG     382
 4  n02056570  n02056570/sketch_40.JPEG      80)

In [3]:
# 데이터의 기본적인 통계 정보를 출력
data_description = train_data.describe(include='all')

# class_name의 unique한 값의 개수를 출력
unique_classes = train_data['class_name'].nunique()

# target의 unique한 값의 개수를 출력
unique_targets = train_data['target'].nunique()

data_description, unique_classes, unique_targets

(       class_name                image_path        target
 count       15021                     15021  15021.000000
 unique        500                     15021           NaN
 top     n04532106  n01872401/sketch_50.JPEG           NaN
 freq           31                         1           NaN
 mean          NaN                       NaN    249.989082
 std           NaN                       NaN    144.471752
 min           NaN                       NaN      0.000000
 25%           NaN                       NaN    125.000000
 50%           NaN                       NaN    250.000000
 75%           NaN                       NaN    375.000000
 max           NaN                       NaN    499.000000,
 500,
 500)

In [4]:
# 학습 데이터의 class, image path, target에 대한 정보가 들어있는 csv파일을 읽기.
train_info = pd.read_csv(traindata_info_file)

# 총 class의 수를 측정.
num_classes = len(train_info['target'].unique())

# 각 class별로 8:2의 비율이 되도록 학습과 검증 데이터를 분리.
train_df, val_df = train_test_split(
    train_info, 
    test_size=0.2,
    stratify=train_info['target']
)

In [5]:
# 학습에 사용할 Transform을 선언.
transform_selector = TransformSelector(
    transform_type = "albumentations"
)
train_transform = transform_selector.get_transform(is_train=True)
val_transform = transform_selector.get_transform(is_train=False)

# 학습에 사용할 Dataset을 선언.
train_dataset = CustomDataset(
    root_dir=traindata_dir,
    info_df=train_df,
    transform=train_transform
)
val_dataset = CustomDataset(
    root_dir=traindata_dir,
    info_df=val_df,
    transform=val_transform
)

# 학습에 사용할 DataLoader를 선언.
train_loader = DataLoader(
    train_dataset, 
    batch_size=64, 
    shuffle=True
)
val_loader = DataLoader(
    val_dataset, 
    batch_size=64, 
    shuffle=False
)

In [6]:
# 학습에 사용할 장비를 선택.
# torch라이브러리에서 gpu를 인식할 경우, cuda로 설정.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [7]:
from PIL import Image
import requests

from transformers import CLIPProcessor, CLIPModel
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
model.to(device)

train_dataset = CLIPDataset(
    root_dir=traindata_dir,
    info_df=train_df,
    transform=train_transform,
    processor=processor,
    device=device
)
inference = CLIPDataset(
    root_dir=traindata_dir,
    info_df=train_df,
    transform=train_transform,
    processor=processor,
    device=device,
    is_inference=True
)
train_loader = DataLoader(
    train_dataset, 
    batch_size=64, 
    shuffle=True
)
mini_values=get_imagenet_ditction(mini=True,values=True)
mini_dict=get_imagenet_ditction(mini=True,values=False)
outputs = model(**train_dataset[3])
logits_per_image = outputs.logits_per_image  # this is the image-text similarity score
probs = logits_per_image.softmax(dim=1)  # we can take the softmax to get the label probabilities
probs



c:\Users\zin\anaconda3\envs\p312\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
c:\Users\zin\anaconda3\envs\p312\Lib\site-packages\transformers\models\clip\modeling_clip.py:480: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


tensor([[1.]], device='cuda:0', grad_fn=<SoftmaxBackward0>)

In [14]:
mini_values

['a photo of echidna, spiny anteater, anteater',
 'a photo of ibex, Capra ibex',
 'a photo of Border collie',
 'a photo of sleeping bag',
 'a photo of king penguin, Aptenodytes patagonica',
 'a photo of mushroom',
 'a photo of West Highland white terrier',
 'a photo of Scotch terrier, Scottish terrier, Scottie',
 'a photo of ox',
 'a photo of torch',
 'a photo of water buffalo, water ox, Asiatic buffalo, Bubalus bubalis',
 'a photo of volcano',
 'a photo of printer',
 'a photo of notebook, notebook computer',
 'a photo of Indian cobra, Naja naja',
 'a photo of Dandie Dinmont, Dandie Dinmont terrier',
 'a photo of wood rabbit, cottontail, cottontail rabbit',
 'a photo of oscilloscope, scope, cathode-ray oscilloscope, CRO',
 'a photo of hammer',
 'a photo of ambulance',
 'a photo of red-breasted merganser, Mergus serrator',
 'a photo of American alligator, Alligator mississipiensis',
 'a photo of white wolf, Arctic wolf, Canis lupus tundrarum',
 'a photo of tailed frog, bell toad, ribbed

In [10]:
all_text_token=processor(text=mini_values,return_tensors="pt", padding=True).to(device)

#train_dataset[0]
all_text_token
all = all_text_token
all['pixel_values'] = inference[0]['pixel_values']
all

{'input_ids': tensor([[49406,   320,  1125,  ..., 49407, 49407, 49407],
        [49406,   320,  1125,  ..., 49407, 49407, 49407],
        [49406,   320,  1125,  ..., 49407, 49407, 49407],
        ...,
        [49406,   320,  1125,  ..., 49407, 49407, 49407],
        [49406,   320,  1125,  ..., 49407, 49407, 49407],
        [49406,   320,  1125,  ..., 49407, 49407, 49407]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]], device='cuda:0'), 'pixel_values': tensor([[[[1.9303, 1.9303, 1.9303,  ..., 1.9303, 1.9303, 1.9303],
          [1.9303, 1.9303, 1.9303,  ..., 1.9303, 1.9303, 1.9303],
          [1.9303, 1.9303, 1.9303,  ..., 1.9303, 1.9303, 1.9303],
          ...,
          [1.9303, 1.9303, 1.9303,  ..., 1.9303, 1.9303, 1.9303],
          [1.9303, 1.9303, 1.9303,  ..., 1.9303, 1.9303, 

In [14]:
outputs = model(**all)
logits_per_image = outputs.logits_per_image  # this is the image-text similarity score
probs = logits_per_image.softmax(dim=1)  # we can take the softmax to get the label probabilities
probs

tensor([[2.4250e-06, 4.7808e-07, 1.4084e-07, 6.1026e-07, 1.2219e-06, 1.5618e-07,
         1.3654e-07, 1.6591e-07, 1.0094e-06, 1.7757e-06, 3.8757e-06, 6.7103e-07,
         4.3323e-06, 1.6178e-06, 1.7982e-08, 4.0184e-10, 1.0608e-06, 6.6154e-09,
         5.4286e-07, 3.7540e-07, 3.2528e-10, 2.4911e-11, 5.8909e-07, 2.4358e-06,
         4.9505e-09, 6.9927e-07, 3.0352e-09, 3.1255e-09, 5.0952e-06, 2.6914e-06,
         3.6494e-05, 2.8568e-06, 5.4938e-08, 2.3865e-07, 5.4340e-07, 1.9723e-07,
         9.8283e-09, 5.5792e-06, 8.3405e-08, 1.0832e-05, 1.0923e-08, 1.1829e-06,
         2.5142e-06, 1.1750e-08, 2.2693e-06, 3.3161e-07, 2.4948e-05, 6.8290e-10,
         1.3408e-06, 3.1438e-08, 2.3221e-07, 2.4480e-07, 2.8320e-06, 6.5495e-08,
         2.4975e-06, 5.5090e-08, 7.0397e-07, 3.3545e-08, 3.4453e-06, 2.8976e-07,
         4.6069e-07, 3.4328e-07, 4.0547e-06, 5.9967e-06, 3.8941e-07, 2.9734e-02,
         3.1275e-06, 5.3336e-09, 1.4300e-05, 2.4754e-07, 1.1967e-07, 9.2541e-08,
         1.1553e-06, 1.8598e

In [15]:
max(probs[0])
p=probs.detach().cpu().numpy()

In [16]:
top_5 = np.argsort(p[0])[-5:]
top_1 = np.argsort(p[0])[-1:]
top_5_text = p[0][top_5]
top_1_text = p[0][top_1]

mini_values_numpy = np.array(mini_values)
#mini_values[mini_values_numpy]
print(mini_values_numpy[top_5])
print(mini_values_numpy[top_1])



['tarantula' 'wolf spider, hunting spider'
 'garden spider, Aranea diademata' 'black widow, Latrodectus mactans'
 'harvestman, daddy longlegs, Phalangium opilio']
['harvestman, daddy longlegs, Phalangium opilio']


In [12]:
# images = [inference[i] for i in range(100)] #len(inference)
# image_input = images#torch.stack(images).to(device)
# text_input = all_text_token.to(device)
len(inference)

12016

In [13]:
!ulimit -Sn

'ulimit' is not recognized as an internal or external command,
operable program or batch file.


In [13]:
images ,classes= inference.get_images(0,4000)

In [14]:
inputs = processor(text=mini_values, images=images, return_tensors="pt", padding=True).to(device)
with torch.no_grad():
    outputs = model(**inputs)

In [16]:
logits_per_image = outputs.logits_per_image  # 이미지와 텍스트 간 유사도
probs = logits_per_image.softmax(dim=-1)  # softmax로 확률 계산
p = probs.cpu()

In [18]:
mini_values_numpy = np.array(mini_values)
count_top1 = 0
count_top5 = 0
for index in range(len(p)):
    top_5 = np.argsort(p[index])[-5:]
    top_1 = np.argsort(p[index])[-1:]
    top_5_text = p[index][top_5]
    top_1_text = p[index][top_1]
    if any(mini_dict[classes[index]] in item for item in mini_values_numpy[top_5]):
        count_top5+=1
    if mini_dict[classes[index]] == mini_values_numpy[top_1]:
        count_top1+=1
        #print(mini_dict[classes[index]])
        #print(mini_values_numpy[top_1])
        #print()

print(f"Top 1 accuracy is {count_top1/len(p)*100:.2f}%")
print(f"Top 5 accuracy is {count_top5/len(p)*100:.2f}%")
print(f"Top 1 count is {count_top1}")
print(f"Top 5 count is {count_top5}")

Top 1 accuracy is 45.92%
Top 5 accuracy is 72.60%
Top 1 count is 1837
Top 5 count is 2904


In [13]:
images ,classes= inference.get_images(4000,8000)

inputs = processor(text=mini_values, images=images, return_tensors="pt", padding=True).to(device)

with torch.no_grad():
    outputs = model(**inputs)
logits_per_image = outputs.logits_per_image  # 이미지와 텍스트 간 유사도
probs = logits_per_image.softmax(dim=-1)  # softmax로 확률 계산
p = probs.cpu()

mini_values_numpy = np.array(mini_values)
count_top1 = 0
count_top5 = 0
for index in range(len(p)):
    top_5 = np.argsort(p[index])[-5:]
    top_1 = np.argsort(p[index])[-1:]
    top_5_text = p[index][top_5]
    top_1_text = p[index][top_1]
    if any(mini_dict[classes[index]] in item for item in mini_values_numpy[top_5]):
        count_top5+=1
    if mini_dict[classes[index]] == mini_values_numpy[top_1]:
        count_top1+=1
        #print(mini_dict[classes[index]])
        #print(mini_values_numpy[top_1])
        #print()

print(f"Top 1 accuracy is {count_top1/len(p)*100:.2f}%")
print(f"Top 5 accuracy is {count_top5/len(p)*100:.2f}%")
print(f"Top 1 count is {count_top1}")
print(f"Top 5 count is {count_top5}")

Top 1 accuracy is 47.52%
Top 5 accuracy is 73.45%
Top 1 count is 1901
Top 5 count is 2938


In [13]:
images ,classes= inference.get_images(8000,12000)

inputs = processor(text=mini_values, images=images, return_tensors="pt", padding=True).to(device)

with torch.no_grad():
    outputs = model(**inputs)
logits_per_image = outputs.logits_per_image  # 이미지와 텍스트 간 유사도
probs = logits_per_image.softmax(dim=-1)  # softmax로 확률 계산
p = probs.cpu()

mini_values_numpy = np.array(mini_values)
count_top1 = 0
count_top5 = 0
for index in range(len(p)):
    top_5 = np.argsort(p[index])[-5:]
    top_1 = np.argsort(p[index])[-1:]
    top_5_text = p[index][top_5]
    top_1_text = p[index][top_1]
    if any(mini_dict[classes[index]] in item for item in mini_values_numpy[top_5]):
        count_top5+=1
    if mini_dict[classes[index]] == mini_values_numpy[top_1]:
        count_top1+=1
        #print(mini_dict[classes[index]])
        #print(mini_values_numpy[top_1])
        #print()

print(f"Top 1 accuracy is {count_top1/len(p)*100:.2f}%")
print(f"Top 5 accuracy is {count_top5/len(p)*100:.2f}%")
print(f"Top 1 count is {count_top1}")
print(f"Top 5 count is {count_top5}")

Top 1 accuracy is 46.65%
Top 5 accuracy is 72.92%
Top 1 count is 1866
Top 5 count is 2917


In [14]:
images ,classes= inference.get_images(12000,12016)

inputs = processor(text=mini_values, images=images, return_tensors="pt", padding=True).to(device)

with torch.no_grad():
    outputs = model(**inputs)
logits_per_image = outputs.logits_per_image  # 이미지와 텍스트 간 유사도
probs = logits_per_image.softmax(dim=-1)  # softmax로 확률 계산
p = probs.cpu()

mini_values_numpy = np.array(mini_values)
count_top1 = 0
count_top5 = 0
for index in range(len(p)):
    top_5 = np.argsort(p[index])[-5:]
    top_1 = np.argsort(p[index])[-1:]
    top_5_text = p[index][top_5]
    top_1_text = p[index][top_1]
    if any(mini_dict[classes[index]] in item for item in mini_values_numpy[top_5]):
        count_top5+=1
    if mini_dict[classes[index]] == mini_values_numpy[top_1]:
        count_top1+=1
        #print(mini_dict[classes[index]])
        #print(mini_values_numpy[top_1])
        #print()

print(f"Top 1 accuracy is {count_top1/len(p)*100:.2f}%")
print(f"Top 5 accuracy is {count_top5/len(p)*100:.2f}%")
print(f"Top 1 count is {count_top1}")
print(f"Top 5 count is {count_top5}")

Top 1 accuracy is 50.00%
Top 5 accuracy is 68.75%
Top 1 count is 8
Top 5 count is 11


In [14]:
count_top1 = 1837 + 1901 + 1866 + 8
count_top5 = 2904 + 2938 + 2917 + 11
print(f"Top 1 accuracy is {count_top1/len(inference)*100:.2f}%")
print(f"Top 5 accuracy is {count_top5/len(inference)*100:.2f}%")
print(f"Top 1 count is {count_top1}")
print(f"Top 5 count is {count_top5}")

Top 1 accuracy is 46.70%
Top 5 accuracy is 72.99%
Top 1 count is 5612
Top 5 count is 8770


In [8]:
images ,classes= inference.get_images(0,4000)

inputs = processor(text=mini_values, images=images, return_tensors="pt", padding=True).to(device)

with torch.no_grad():
    outputs = model(**inputs)
logits_per_image = outputs.logits_per_image  # 이미지와 텍스트 간 유사도
probs = logits_per_image.softmax(dim=-1)  # softmax로 확률 계산
p = probs.cpu()

mini_values_numpy = np.array(mini_values)
count_top1 = 0
count_top5 = 0
for index in range(len(p)):
    top_5 = np.argsort(p[index])[-5:]
    top_1 = np.argsort(p[index])[-1:]
    top_5_text = p[index][top_5]
    top_1_text = p[index][top_1]
    if any(mini_dict[classes[index]] in item for item in mini_values_numpy[top_5]):
        count_top5+=1
    if mini_dict[classes[index]] == mini_values_numpy[top_1]:
        count_top1+=1
        #print(mini_dict[classes[index]])
        #print(mini_values_numpy[top_1])
        #print()

print(f"Top 1 accuracy is {count_top1/len(p)*100:.2f}%")
print(f"Top 5 accuracy is {count_top5/len(p)*100:.2f}%")
print(f"Top 1 count is {count_top1}")
print(f"Top 5 count is {count_top5}")

Top 1 accuracy is 48.05%
Top 5 accuracy is 74.70%
Top 1 count is 1922
Top 5 count is 2988


In [8]:
images ,classes= inference.get_images(4000,8000)

inputs = processor(text=mini_values, images=images, return_tensors="pt", padding=True).to(device)

with torch.no_grad():
    outputs = model(**inputs)
logits_per_image = outputs.logits_per_image  # 이미지와 텍스트 간 유사도
probs = logits_per_image.softmax(dim=-1)  # softmax로 확률 계산
p = probs.cpu()

mini_values_numpy = np.array(mini_values)
count_top1 = 0
count_top5 = 0
for index in range(len(p)):
    top_5 = np.argsort(p[index])[-5:]
    top_1 = np.argsort(p[index])[-1:]
    top_5_text = p[index][top_5]
    top_1_text = p[index][top_1]
    if any(mini_dict[classes[index]] in item for item in mini_values_numpy[top_5]):
        count_top5+=1
    if mini_dict[classes[index]] == mini_values_numpy[top_1]:
        count_top1+=1
        #print(mini_dict[classes[index]])
        #print(mini_values_numpy[top_1])
        #print()

print(f"Top 1 accuracy is {count_top1/len(p)*100:.2f}%")
print(f"Top 5 accuracy is {count_top5/len(p)*100:.2f}%")
print(f"Top 1 count is {count_top1}")
print(f"Top 5 count is {count_top5}")

Top 1 accuracy is 46.85%
Top 5 accuracy is 74.62%
Top 1 count is 1874
Top 5 count is 2985


In [11]:
images ,classes= inference.get_images(8000,12000)

inputs = processor(text=mini_values, images=images, return_tensors="pt", padding=True).to(device)

with torch.no_grad():
    outputs = model(**inputs)
logits_per_image = outputs.logits_per_image  # 이미지와 텍스트 간 유사도
probs = logits_per_image.softmax(dim=-1)  # softmax로 확률 계산
p = probs.cpu()

mini_values_numpy = np.array(mini_values)
count_top1 = 0
count_top5 = 0
for index in range(len(p)):
    top_5 = np.argsort(p[index])[-5:]
    top_1 = np.argsort(p[index])[-1:]
    top_5_text = p[index][top_5]
    top_1_text = p[index][top_1]
    if any(mini_dict[classes[index]] in item for item in mini_values_numpy[top_5]):
        count_top5+=1
    if mini_dict[classes[index]] == mini_values_numpy[top_1]:
        count_top1+=1
        #print(mini_dict[classes[index]])
        #print(mini_values_numpy[top_1])
        #print()

print(f"Top 1 accuracy is {count_top1/len(p)*100:.2f}%")
print(f"Top 5 accuracy is {count_top5/len(p)*100:.2f}%")
print(f"Top 1 count is {count_top1}")
print(f"Top 5 count is {count_top5}")

Top 1 accuracy is 47.75%
Top 5 accuracy is 75.17%
Top 1 count is 1910
Top 5 count is 3007


In [15]:
images ,classes= inference.get_images(12000,12016)

inputs = processor(text=mini_values, images=images, return_tensors="pt", padding=True).to(device)

with torch.no_grad():
    outputs = model(**inputs)
logits_per_image = outputs.logits_per_image  # 이미지와 텍스트 간 유사도
probs = logits_per_image.softmax(dim=-1)  # softmax로 확률 계산
p = probs.cpu()

mini_values_numpy = np.array(mini_values)
count_top1 = 0
count_top5 = 0
for index in range(len(p)):
    top_5 = np.argsort(p[index])[-5:]
    top_1 = np.argsort(p[index])[-1:]
    top_5_text = p[index][top_5]
    top_1_text = p[index][top_1]
    if any(mini_dict[classes[index]] in item for item in mini_values_numpy[top_5]):
        count_top5+=1
    if mini_dict[classes[index]] == mini_values_numpy[top_1]:
        count_top1+=1
        #print(mini_dict[classes[index]])
        #print(mini_values_numpy[top_1])
        #print()

print(f"Top 1 accuracy is {count_top1/len(p)*100:.2f}%")
print(f"Top 5 accuracy is {count_top5/len(p)*100:.2f}%")
print(f"Top 1 count is {count_top1}")
print(f"Top 5 count is {count_top5}")

Top 1 accuracy is 56.25%
Top 5 accuracy is 75.00%
Top 1 count is 9
Top 5 count is 12


In [12]:
print("only text")
count_top1 = 1837 + 1901 + 1866 + 8
count_top5 = 2904 + 2938 + 2917 + 11
print(f"Top 1 accuracy is {count_top1/len(inference)*100:.2f}%")
print(f"Top 5 accuracy is {count_top5/len(inference)*100:.2f}%")
print(f"Top 1 count is {count_top1}")
print(f"Top 5 count is {count_top5}")
print()
print("a photo of text")
count_top1_a = 1922 + 1874 + 1910 + 9
count_top5_a = 2988 + 2985 + 3007 + 12
print(f"Top 1 accuracy is {count_top1_a/len(inference)*100:.2f}%")
print(f"Top 5 accuracy is {count_top5_a/len(inference)*100:.2f}%")
print(f"Top 1 count is {count_top1_a}")
print(f"Top 5 count is {count_top5_a}")
print()
print(f"Top 1 accuracy is {(count_top1_a-count_top1)/len(inference)*100:.2f}%")
print(f"Top 5 accuracy is {(count_top5_a-count_top5)/len(inference)*100:.2f}%")
print(f"Top 1 count is {count_top1_a-count_top1}")
print(f"Top 5 count is {count_top5_a-count_top5}")


only text
Top 1 accuracy is 46.70%
Top 5 accuracy is 72.99%
Top 1 count is 5612
Top 5 count is 8770

a photo of text
Top 1 accuracy is 47.56%
Top 5 accuracy is 74.83%
Top 1 count is 5715
Top 5 count is 8992

Top 1 accuracy is 0.86%
Top 5 accuracy is 1.85%
Top 1 count is 103
Top 5 count is 222
